# Programming in lower level

* Sometimes a control engineer may want to make computers to give commands to or read measurements from machines.



* Understanding how computers work *under the hood* would be helpful in such situations.



## Chinese Room

* Prof. John Searle of UC Berkeley published in 1980.

[![Chinese Room](https://cdn-images-1.medium.com/max/1600/1*ve1L0BoTEEChNZMwn7_zig.jpeg)](https://medium.com/@transphilosophr/searles-chinese-room-thought-experiment-a-twist-c7eb28f65e6c)

* Assume a person inside of a closed room with a slots for questions and answers.

* From outside, a person would (in)put a letter with a question in an unfamiliar foreign language into the room.

* The person inside of the room has a very big book with answers to all questions of that foreign language.

* The person inside of the room would lookup the answer from the book and (out)put another letter in the foreign language with the answer out of the room.

* Now, do people outside of the room can tell if the person inside knows the foreign language?

### Analogy

* Let's compare this room with a computer.

| thought experiment | computer |
|:------:|:------:|
| room | CPU |
| person inside of the room | CPU electronics |
| book in the room | CPU instruction set |
| people outside of the room | memory |
| letter coming in | machine language instruction |
| letter going out | result of the machine language instruction |
| frequency of letter | clock speed |



## Classes of computer memories

| memory | physical location | amount | speed | purpose |
|:-----:|:-----:|:-----:|:-----:|:-----:|
| registers | CPU | very limited | fastest | operation, addressing, I/O |
| cache | In or close to CPU | not much | faster | accelerating information fetching |
| main memory | motherboard RAM | GBs nowadays | fast | store code and data |
| stack | within RAM | limited | RAM | store function call information |
| [heap](https://en.wikipedia.org/wiki/Memory_management#HEAP) | within RAM | less limited | RAM | allocate memory from here |
| SSD | SSD | TBs? | [Flash memory](https://en.wikipedia.org/wiki/Flash_memory) speed | preserve code and data  |
| HDD | HDD | TBs | Magnetic disk speed | preserve code and data  |
| Memory mapped I/O | ??? | limited | ??? | communicate with a device  |



* Following [figure](https://thenextweb.com/shareables/2011/12/26/this-is-what-a-5mb-hard-drive-looked-like-is-1956-required-a-forklift/) shows an eary [hard disk drive](https://en.wikipedia.org/wiki/IBM_305_RAMAC) of 5 Megabytes in 1950s.

![IBM HDD 5MB 1956](https://cdn0.tnwcdn.com/wp-content/blogs.dir/1/files/2011/12/Screen-Shot-2011-12-26-at-18.38.18.png)


* Among these memories, cache is usually not accessable from the program.

* Disk drives, programs can access through file systems.

* Main memory (+ stack and heap) is *addressable*; thus we can use *pointer*s to access.

* How to access registers depends on the compilers.

* Memory mapped IO is addressable; thus we can use pointers to access.  However, if output, the computer would send what we write here to a device, possibly an external one.

## Working with bits

* To utilize certain features of microcontrollers, often times we need to turn on or off certain bits of certain memory location.

* The pointer of C/C++ is a suitable tool for this type of task.

* However, the smallest unit of memory that we can change is a byte; hence we may need to first read the current status of the particular byte, change that one bit, and then write the *bit pattern* to that byte.

### Revisiting `struct` and `union`

* `struct` is a group of data.  An array is also a group of data.  Difference is, to access data in `struct`, we would use *field name*s; for arrays, indices.

* Also, *field*s of struct may have different types; for arrays, all same types.

* Fields of `unions` share the same memory.

#### `union`: Little endian vs Big endian

* Let's see how our computer stores a multi-byte integer.

* Using `union` we can make a multi-byte integer and an array of characters share the memory.

In [ ]:
%%writefile little_or_big.cpp
#include <cstdio>
#include <cstdint>

// i and c[4] would share the same memory
union little_big_tag{
    uint32_t i;
    char c[4];
};


int32_t main(const int32_t argn, const char * argv[]){
    
    // an instance of the union
    union little_big_tag s;

    // See if both int and char[] member share the same memory
    printf("&(s.i) = %08lx\n", (uint64_t) (&(s.i)));
    printf("&(s.c) = %08lx\n", (uint64_t) (&(s.c)));
    
    printf(
        "They share the same memory if non-zero: %x\n", 
        (
            (uint64_t) (&(s.i)) == (uint64_t) (&(s.c))
        )
    );

    // assign an integer to the integer member of s
    s.i = 0x01020304;
    // reading the memory as one 32 bit integer
    printf("s.i = %08x\n", s.i);
    // as four 8 bit characters
    printf("s.c = %02x %02x %02x %02x\n", s.c[0], s.c[1], s.c[2], s.c[3]);

    // check address of each member of char[4]
    // which part of integer is in which part of array?
    int32_t i = 0;
    for (i = 0; 4 > i; i++){
        printf("&(s.c[%d]) = %08lx\n", i, (uint64_t) (&(s.c[i])));
    }
}



In [ ]:
%%bash
# Detect OS type because OSX may need different options
# https://stackoverflow.com/questions/3466166/how-to-check-if-running-in-cygwin-mac-or-linux/18790824
unameOut="$(uname -s)"
case "${unameOut}" in
    Linux*)     machine=Linux;;
    Darwin*)    machine=Mac;;
    CYGWIN*)    machine=Cygwin;;
    MINGW*)     machine=MinGw;;
    *)          machine="UNKNOWN:${unameOut}"
esac


if [ $machine == "Linux" ]; then
    g++ -Wall -g little_or_big.cpp -o ./little_or_big -Wa,-adhln=little_or_big.s
elif [ "Mac" == $machine ]; then
    # https://stackoverflow.com/questions/10990018/
    clang++ -S -mllvm --x86-asm-syntax=intel little_or_big.cpp
    clang++ -Wall -g little_or_big.cpp -o little_or_big
else
    g++ -Wall -g little_or_big.cpp -o ./little_or_big.s -S
    g++ -Wall -g little_or_big.cpp -o ./little_or_big
fi



In [ ]:
# https://stackoverflow.com/questions/4760215/running-shell-command-from-python-and-capturing-the-output
# https://stackoverflow.com/questions/35160256/how-do-i-output-lists-as-a-table-in-jupyter-notebook
import subprocess
import IPython.display as disp

result = subprocess.run(['./little_or_big'], stdout=subprocess.PIPE)

print(result.stdout.decode())



In [ ]:
%%bash

rm little_or_big.cpp little_or_big



#### 16bit color using `struct` and `union`

* For each field of struct, we may specify its length in **bits**.

* Followinging example shows a 16bit color information.

``` C++
#include <cstdint>

// Josh Kunz, Bit-field Packing in GCC and Clang, https://jkz.wtf/bit-field-packing-in-gcc-and-clang
struct high_color_tag{
    uint16_t red:5;
    uint16_t green:6;
    uint16_t blue:5;
};
```



* We can see that the sum of bits is `5 + 6 + 5 = 16`; two bytes.

* `struct` would allocate `red`, `green`, and `blue` values as follows.

In [ ]:
# https://stackoverflow.com/questions/35160256/how-do-i-output-lists-as-a-table-in-jupyter-notebook
# http://nbviewer.jupyter.org/github/ipython/ipython/blob/4.0.x/examples/IPython%20Kernel/Rich%20Output.ipynb

import IPython.display as disp

# number of bits
n = 16
nr = 5
ng = 6
nb = 5

disp.display(
    disp.Markdown(
        '\n'.join(
            [
                ' | '.join(str(k) for k in range(n-1, 0-1, -1)),
                '|'.join(':---:' for k in range(1, n+1)),
                ' | '.join(['`b`']*nb + ['`g`']*ng + ['`r`']*nr),
            ],
        )
    )
)



* Following C++ code would show an example.

In [ ]:
%%writefile bit_field_struct.cpp

#include <bitset>
#include <cstdint>
#include <cstdlib>
#include <iomanip>
#include <iostream>


// Josh Kunz, Bit-field Packing in GCC and Clang, https://jkz.wtf/bit-field-packing-in-gcc-and-clang
struct high_color_tag{
    uint16_t red:5;
    uint16_t green:6;
    uint16_t blue:5;
};


union high_color_union_tag{
    struct high_color_tag rgb_struct;
    uint16_t hex;
};


int32_t main(const int32_t argn, const char * argv[]){
    union high_color_union_tag rgb_union;
    const int32_t n = 10;

    int32_t r = 0, g = 0, b = 0, i = 0;
    
    std::cout << "| `r` | `g` | `b` | `hex` |" << '\n';
    std::cout << "|:---:|:---:|:---:|:-----:|" << '\n';

    for(i = 0; n > i; ++i){
        r = rand() % (1 << 5);
        g = rand() % (1 << 6);
        b = rand() % (1 << 5);

        rgb_union.rgb_struct.red = r;
        rgb_union.rgb_struct.green = g;
        rgb_union.rgb_struct.blue = b;

        // https://stackoverflow.com/questions/7349689
        std::bitset<5> b_r(r);
        std::bitset<6> b_g(g);
        std::bitset<6> b_b(b);
        std::bitset<16> b_hex(rgb_union.hex);

        std::cout << "| " << std::hex << std::setw(6) << b_r
                    << " | " << std::setw(6) << b_g
                    << " | " << std::setw(6) << b_b
                    << " | " << std::setw(16) << b_hex
                    << " |\n";
    }

    // std::cout << "sizeof(rgb_union.rgb_struct)" << sizeof(rgb_union.rgb_struct) << '\n';

}



In [ ]:
%%bash
# Detect OS type because OSX may need different options
# https://stackoverflow.com/questions/3466166/how-to-check-if-running-in-cygwin-mac-or-linux/18790824
unameOut="$(uname -s)"
case "${unameOut}" in
    Linux*)     machine=Linux;;
    Darwin*)    machine=Mac;;
    CYGWIN*)    machine=Cygwin;;
    MINGW*)     machine=MinGw;;
    *)          machine="UNKNOWN:${unameOut}"
esac


if [ $machine == "Linux" ]; then
    g++ -Wall -g bit_field_struct.cpp -o ./bit_field_struct -Wa,-adhln=bit_field_struct.s
elif [ "Mac" == $machine ]; then
    # https://stackoverflow.com/questions/10990018/
    clang++ -S -mllvm --x86-asm-syntax=intel bit_field_struct.cpp
    clang++ -Wall -g bit_field_struct.cpp -o bit_field_struct
else
    g++ -Wall -g bit_field_struct.cpp -o ./bit_field_struct.s -S
    g++ -Wall -g bit_field_struct.cpp -o ./bit_field_struct
fi



In [ ]:
# https://stackoverflow.com/questions/4760215/running-shell-command-from-python-and-capturing-the-output
# https://stackoverflow.com/questions/35160256/how-do-i-output-lists-as-a-table-in-jupyter-notebook
import subprocess
import IPython.display as disp

result = subprocess.run(['./bit_field_struct'], stdout=subprocess.PIPE)

disp.display(disp.Markdown(result.stdout.decode()))



In [ ]:
%%bash

rm bit_field_struct.cpp bit_field_struct

